In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 10
Maxiterations_turbine = 10

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]


time_ac = 3600
PKM_zaryad = True
PKM_razryad = False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 20
pkm_pgu_tol = 10**-1

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
]


########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


########РАСЧЕТ расхода пара на ПКМ из условия заполнения хранилища################
########РАСЧЕТ мощности ГТУ из условия тепловой мощности и ################

################Расчет минимальной нагрузки ГТУ при остальных нормальных условиях############

Max_iterations_minimum = 10

if Сalculate_minimum == True:
    n_GTU = GTU_input.at["n", 1]
    start_time = time.time()
    for i in range(Max_iterations_minimum):
        if i < 6:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                Iter_pkm,
            ) = (2, 2, 15, 3)
        else:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                Iter_pkm,
            ) = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
                Iter_pkm,
            )

        Delt_Gcnd = 100
        Delt_Nturb = 100
        Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
        Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
        Delt_Gvd = (
            water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
            - 0.25
        ) / 0.25
        Delt_Gnd = (
            water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
            - 0.5
        ) / 0.5
        Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd)
        if n_GTU == 1 and Delta_min < 0:
            print("Мощность ГТУ 100% и расход пара все еще слишком мал")
        n_GTU = n_GTU - Delta_min / 8
        GTU_input.at["n", 1] = n_GTU

        arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
            New_iterations_KU_TU,
            New_iterations_cotel,
            New_iterations_turbine,
        )
        gas_streams = Calculate_CCGT_PKM_iter(arguments_all_it, Iter_pkm, pkm_pgu_tol)
        print(
            f"Время {i+1} итерации расчета мощности ГТУ при ПГУ с ПКМ: --- {round((time.time() - start_time), 1)} сек. ---"
        )
        print(f"Отклонение от ограничения минимальное равно {Delta_min}")
        print(f"n_GTU: {n_GTU}")
        if abs(Delta_min) < Calctolerance:
            gas_streams = Calculate_CCGT_PKM_iter(
                arguments_all_it, Iter_pkm, pkm_pgu_tol
            )
            print(f"Отклонение от ограничения минимальное равно {Delta_min}")
            print(f"Относительная мощность ГТУ равна {n_GTU}")
            print(
                f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
            )
            break
        if i == Max_iterations_minimum - 1:
            print(
                "Достигнуто максимальное количество итераций минимального расхода в ПГУ",
                i + 1,
            )

Gst 63.4 Gst 78
8
5067.193106343894 6999.6623390577815 15964.811016832335 19072.879196799302
-ТЕПЛА ОТ ПКМ НЕ ХВАТАЕТ НА ТЕПЛОФИКАЦИЮ 47104.54565903331 / 167682.95067204515
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  11.0 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Ошибка!!! при температуре 0.0
Ошибка!!! при температуре 0.0
Ошибка!!! при температуре 0.0
Ошибка!!! при температуре 0.0
Достигнуто максимальное количество итераций давления верхнего отбора
Расход в конденсатор равен нулю
Расход из турбины G: 48.272058483027934
Расход в ГПК G: 29.69548812054422
Достигнуто максимальное количество итераций котла-утилизатора:2
Расход в конденсатор меньше допустимого (4,44 кг/с)
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 2 итерации расчета КУ+ТУ:---  27.2 сек. ---


In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,535.168535,0.1,950.657124,868.647487,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,10.532257,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,538.700616,0.1,957.610905,879.179745,0.779442,0.122305,0.030841,0.060886,0.006526
PEVD-IVD,466.449114,0.1,875.028322,879.179745,0.779442,0.122305,0.030841,0.060886,0.006526
IVD-EVD,294.189242,0.1,683.699101,879.179745,0.779442,0.122305,0.030841,0.060886,0.006526
EVD-PPND,228.282259,0.1,612.475944,879.179745,0.779442,0.122305,0.030841,0.060886,0.006526
PPND-IND,225.917000,0.1,609.938211,879.179745,0.779442,0.122305,0.030841,0.060886,0.006526
IND-GPK,176.457709,0.1,557.136311,879.179745,0.779442,0.122305,0.030841,0.060886,0.006526
GPK-out,115.178725,0.1,492.331836,879.179745,0.779442,0.122305,0.030841,0.060886,0.006526


In [3]:
water_streams

,T,P,H,G,S,X
AIR,0.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,500.686574,4.914785,3436.981175,109.749287,NaN,NaN
IVD-PEVD,279.640318,6.381842,2780.275240,109.749287,NaN,NaN
EVD-IVD,279.640318,6.381842,1261.500222,109.749287,NaN,NaN
PEN-EVD,164.129695,6.381842,696.704141,109.749287,NaN,NaN
BND-PEN,163.085018,0.668283,688.891615,109.749287,NaN,NaN
PPND-DROSND,202.495620,0.653460,2853.287877,23.856189,NaN,NaN
IND-PPND,163.085017,0.668283,2760.785186,23.856189,NaN,NaN
GPK-IND,163.085018,0.668283,714.898694,133.605475,NaN,NaN
GPK-REC,163.085018,0.668283,714.898694,133.628304,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3683.821929,10.602461,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3023.629730,10.602461,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.0,2.0,2545.703624,10.602461,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.0,2.0,746.793127,10.602461,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.0,2.0,1345.405927,4.221108,0.0,0.0,0.168570,0.0,0.051503,0.097617,0.680303,0.002007
SGaccum-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SGaccum-GTU,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COMP-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COMB-TURB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
electric

,Ni,N,KPDm,KPD
PEN,0.857419,0.788295,1.087688,0.807400
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,328.567569,NaN,-819.643471
DK,NaN,6.647035,NaN,NaN
Turbine,15.236909,NaN,NaN,NaN
Tots1,7.151930,NaN,NaN,0.818894
Tots2,6.491928,NaN,NaN,0.828047
Tots3,1.593051,NaN,NaN,0.810567
Tots4,0.000000,NaN,NaN,0.000000


In [6]:
heaters

,Qw,Qg,KPD
PEVD,71878.885201,72604.934546,0.99
IVD,166530.647882,168212.775638,0.99
EVD,61991.777755,62617.957328,0.99
PPND,2208.812472,2231.123709,0.99
IND,45958.136604,46422.360206,0.99
GPK,56405.034496,56974.782320,0.99
SP2,25.586222,NaN,0.99
SP1,95.580611,NaN,0.99
OD,8.017332,NaN,0.99
Strans,283904.840282,NaN,NaN
